In [1]:
#Required libraries and functions
import pandas as pd
import numpy as np
from scipy.stats import rankdata
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt

In [28]:
#Measure of similarity to OR operator

def orness(w_ord):

    '''
    Function to measure orness according to original Yager,1988 formula
    ORDER WEIGHTS SHOULD BE IMPUTED IN DESCENDING ORDER
    such that w_ord[0] is the weight associate with the MAX
    and w_ord[n] is the weight associated with the MIN
    '''

    w_ord = np.array(w_ord)
    n = len(w_ord)

    coef = np.arange(n-1,-1,-1)
    value = w_ord.dot(coef)/(n-1)

    return value

In [1]:
#OWA: naive version - Yager, 1988

def owa(data:[pd.Series, pd.DataFrame, np.array], w_imp, w_ord) -> np.array:    #as data, only pd.Series, pd.DataFrames and np.arrays are accepted; w_int, w_fin: any mono-dimensional iterable.

    '''
    Function to compute ordered weighted average of a dataset 
    composed of n. observation*n. criteria,
    data = pd.Series/pd.DataFrame
    w_int = importance weights
    w_ord = order weights
    ORDER WEIGHTS SHOULD BE IMPUTED IN DESCENDING ORDER
    such that w_ord[0] is the weight associate with the MAX
    and w_ord[n] is the weight associated with the MIN,
    if no importance weights,it is sufficient to set w_imp = 1
    '''

    w_ord = np.flip(w_ord) #This is required to be aligned with next OWA computations since numpy sorts only in ascending order


    w_imp = np.array(w_imp)
    w_ord = np.array(w_ord)

    if type(data).__module__ != np.__name__:
       data = data.to_numpy()

    int_res = data * w_imp # intermediate results = importance weights x raw values
    int_res = np.sort(int_res, axis = 1) # sort intermediate results in ascending order
    values = int_res.dot(w_ord) # final product with order weights and sum

    return int_res

owa(test_dta, test_w_imp, test_w_ord.loc[att,:])

NameError: name 'pd' is not defined

In [37]:
#WOWA - Torra, 1998

def  wowa(data:[pd.Series, pd.DataFrame, np.array], w_imp, w_ord) -> np.array:    #as data, only pd.Series, pd.DataFrames and np.arrays are accepted; w_imp, w_ord: any mono-dimensional iterable

    '''
    Function to compute WOWA of a dataset 
    composed of n. observation*n. criteria,
    the interpolation function chosen is linear interpolation,
    data = pd.Series/pd.DataFrame
    w_int = importance weights
    w_fin = order weights
    ORDER WEIGHTS SHOULD BE IMPUTED IN DESCENDING ORDER, 
    such that w_ord[0] is the weight associate with the MAX
    and w_ord[n] is the weight associated with the MIN
    '''

    w_imp = np.array(w_imp)
    w_ord = np.array(w_ord)

    if type(data).__module__ != np.__name__:
        data = data.to_numpy()

    w_ord_cum = np.cumsum(np.insert(w_ord,0,0))
    points = np.arange(0,w_ord.shape[0]+1)/w_ord.shape[0]
    
    #To ensure that the function is robust to numerical errors (i.e. rounding)
    w_ord_cum = np.insert(w_ord_cum, len(w_ord_cum), 1)
    points = np.insert(points, len(points), 1.1)
        
    #Compute the interpolation function needed to determine the final weights, note that w_ord is in still in "descending order" here
    int_fun = interp1d(points, w_ord_cum)
    int_fun_vect = np.vectorize(int_fun) #vectorize function, to enable its application to np.arrays

    #sort the importance weights array according to the order of the values ("observation specific")
    w_imp_long = np.full((data.shape[0],data.shape[1]),w_imp)
    sorted_ind = np.argsort(data, axis = 1) #Automatically sorts in ascending order
    w_imp_long = np.take_along_axis(w_imp_long,sorted_ind,axis=1) 

    #Compute the final weights applying linear interpolation
    quant_fun = int_fun_vect(np.cumsum(np.flip(w_imp_long,1), axis = 1))  #This operations should be done with importance weights sorted according to observation values, but in descending order (np.flip())
    w_fin = np.diff(quant_fun, axis = 1, prepend = 0)

    #multiply the final weights (different for every observation) for the values, and compute the final sum
    values = (np.take_along_axis(data,sorted_ind,axis=1)*np.flip(w_fin,1)).sum(axis=1)  #np.take_along_axis(data.to_numpy(),sorted_ind,axis=1) is the same as np.sort(data.to_numpy())
    
    return values 


array([0.166, 0.35 , 0.18 , 0.09 ])

In [36]:
#WOWA - Torra, 1998 (optimized version, if n. unique(weights) << n. of dataset rows)


def  wowa(data:[pd.Series, pd.DataFrame, np.array], w_imp, w_ord) -> np.array:    #as data, only pd.Series, pd.DataFrames and np.arrays are accepted; w_imp, w_ord: any mono-dimensional iterable

    '''
    Function to compute WOWA of a dataset 
    composed of n. observation*n. criteria,
    the interpolation function chosen is linear interpolation,
    data = pd.Series/pd.DataFrame
    w_int = importance weights
    w_fin = order weights
    ORDER WEIGHTS SHOULD BE IMPUTED IN DESCENDING ORDER, 
    such that w_ord[0] is the weight associate with the MAX
    and w_ord[n] is the weight associated with the MIN
    '''

    w_imp = np.array(w_imp)
    w_ord = np.array(w_ord) 

    if type(data).__module__ != np.__name__:
        data = data.to_numpy()

    w_ord_cum = np.cumsum(np.insert(w_ord,0,0))
    points = np.arange(0,w_ord.shape[0]+1)/w_ord.shape[0]
    
    #To ensure that the function is robust to numerical errors (i.e. rounding)
    w_ord_cum = np.insert(w_ord_cum, len(w_ord_cum), 1)
    points = np.insert(points, len(points), 1.1)
        
    #Compute the interpolation function needed to determine the final weights, note that w_ord is in still in "descending order" here
    int_fun = interp1d(points, w_ord_cum)
    int_fun_vect = np.vectorize(int_fun) #vectorize function, to enable its application to np.arrays

    #sort the importance weights array according to the order of the values ("observation specific")
    w_imp_long = np.full((data.shape[0],data.shape[1]),w_imp)
    sorted_ind = np.argsort(data, axis = 1) #Automatically sorts in ascending order
    w_imp_long = np.take_along_axis(w_imp_long,sorted_ind,axis=1) 

    #Compute the final weights applying linear interpolation (to all the combinations of cumulative sum of weights in the dataset)
    unique_comb, indices = np.unique(np.cumsum(np.flip(w_imp_long,1),axis = 1),axis=0, return_inverse = True) # computes the interpolation function for all the possible combinations of cumulative sums of weights in the dataset
    quant_fun = int_fun_vect(unique_comb)                                                                     #This operations should be done with importance weights sorted according to observation values, but in descending order (np.flip())
    w_fin = np.diff(quant_fun, axis = 1, prepend = 0)
    w_fin = w_fin[indices] #attributes the right final weights to the whole dataset, reconstructing the array 

    #multiply the final weights (different for every observation) for the values, and compute the final sum
    values = (np.take_along_axis(data,sorted_ind,axis=1)*np.flip(w_fin,1)).sum(axis=1)  #np.take_along_axis(data.to_numpy(),sorted_ind,axis=1) is the same as np.sort(data.to_numpy())
    
    return values



array([0.166, 0.35 , 0.18 , 0.09 ])